In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import firwin
from tqdm import tqdm
from portiloopml.portiloop_python.ANN.wamsley_utils import detect_lacourse
import os

In [3]:


def shift_numpy(arr, num, fill_value=np.nan):
    result = np.empty_like(arr)
    if num > 0:
        result[:num] = fill_value
        result[num:] = arr[:-num]
    elif num < 0:
        result[num:] = fill_value
        result[:num] = arr[-num:]
    else:
        result[:] = arr
    return result

class FIR:
    def __init__(self, nb_channels, coefficients, buffer=None):
        
        self.coefficients = np.expand_dims(np.array(coefficients), axis=1)
        self.taps = len(self.coefficients)
        self.nb_channels = nb_channels
        self.buffer = np.array(buffer) if buffer is not None else np.zeros((self.taps, self.nb_channels))
    
    def filter(self, x):
        self.buffer = shift_numpy(self.buffer, 1, x)
        filtered = np.sum(self.buffer * self.coefficients, axis=0)
        return filtered

    
class FilterPipeline:
    def __init__(self,
                 nb_channels,
                 sampling_rate,
                 power_line_fq=60,
                 use_custom_fir=False,
                 custom_fir_order=20,
                 custom_fir_cutoff=30,
                 alpha_avg=0.1,
                 alpha_std=0.001,
                 epsilon=0.000001,
                 filter_args=[]):
        if len(filter_args) > 0:
            use_fir, use_notch, use_std = filter_args
        else:
            use_fir=True,
            use_notch=False,
            use_std=True
        self.use_fir = use_fir
        self.use_notch = use_notch
        self.use_std = use_std
        self.nb_channels = nb_channels
        assert power_line_fq in [50, 60], f"The only supported power line frequencies are 50 Hz and 60 Hz"
        if power_line_fq == 60:
            self.notch_coeff1 = -0.12478308884588535
            self.notch_coeff2 = 0.98729186796473023
            self.notch_coeff3 = 0.99364593398236511
            self.notch_coeff4 = -0.12478308884588535
            self.notch_coeff5 = 0.99364593398236511
        else:
            self.notch_coeff1 = -0.61410695998423581
            self.notch_coeff2 =  0.98729186796473023
            self.notch_coeff3 = 0.99364593398236511
            self.notch_coeff4 = -0.61410695998423581
            self.notch_coeff5 = 0.99364593398236511
        self.dfs = [np.zeros(self.nb_channels), np.zeros(self.nb_channels)]
        
        self.moving_average = None
        self.moving_variance = np.zeros(self.nb_channels)
        self.ALPHA_AVG = alpha_avg
        self.ALPHA_STD = alpha_std
        self.EPSILON = epsilon
        
        if use_custom_fir:
            self.fir_coef = firwin(numtaps=custom_fir_order+1, cutoff=custom_fir_cutoff, fs=sampling_rate)
        else:
            self.fir_coef = [
                0.001623780150148094927192721215192250384,
                0.014988684599373741992978104065059596905,
                0.021287595318265635502275046064823982306,
                0.007349500393709578957568417933998716762,
                -0.025127515717112181709014251396183681209,
                -0.052210507359822452833064687638398027048,
                -0.039273839505489904766477593511808663607,
                0.033021568427940004020193498490698402748,
                0.147606943281569008563636202779889572412,
                0.254000252034505602516389899392379447818,
                0.297330876398883392486283128164359368384,
                0.254000252034505602516389899392379447818,
                0.147606943281569008563636202779889572412,
                0.033021568427940004020193498490698402748,
                -0.039273839505489904766477593511808663607,
                -0.052210507359822452833064687638398027048,
                -0.025127515717112181709014251396183681209,
                0.007349500393709578957568417933998716762,
                0.021287595318265635502275046064823982306,
                0.014988684599373741992978104065059596905,
                0.001623780150148094927192721215192250384]
        self.fir = FIR(self.nb_channels, self.fir_coef)
        
    def filter(self, value):
        """
        value: a numpy array of shape (data series, channels)
        """
        for i, x in enumerate(value):  # loop over the data series
            # FIR:
            if self.use_fir:
                x = self.fir.filter(x)
            # notch:
            if self.use_notch:
                denAccum = (x - self.notch_coeff1 * self.dfs[0]) - self.notch_coeff2 * self.dfs[1]
                x = (self.notch_coeff3 * denAccum + self.notch_coeff4 * self.dfs[0]) + self.notch_coeff5 * self.dfs[1]
                self.dfs[1] = self.dfs[0]
                self.dfs[0] = denAccum
            # standardization:
            if self.use_std:
                if self.moving_average is not None:
                    delta = x - self.moving_average
                    self.moving_average = self.moving_average + self.ALPHA_AVG * delta
                    self.moving_variance = (1 - self.ALPHA_STD) * (self.moving_variance + self.ALPHA_STD * delta**2)
                    moving_std = np.sqrt(self.moving_variance)
                    x = (x - self.moving_average) / (moving_std + self.EPSILON)
                else:
                    self.moving_average = x
            value[i] = x
        return value

In [6]:
def online_detrend(y, alpha=0.95):
    detrended_y = np.zeros_like(y)
    trend = 0
    for i in range(len(y)):
        trend = alpha * trend + (1 - alpha) * y[i]
        detrended_y[i] = y[i] - trend
    return detrended_y

In [7]:
def raw2filtered(raw):
    '''
    Take in the raw data and filter it online, detrend it
    '''
    filtering4lac = FilterPipeline(nb_channels=1, sampling_rate=250, filter_args=[True, True, False])
    filtered4lac = []
    print(f'Filtering Data Online')
    for i in tqdm(raw):
        filtered4lac.append(filtering4lac.filter(np.array([i])))

    print(f'Detrending Data')
    detrended_data = online_detrend(np.array(filtered4lac).flatten())

    print(f"Running Lacourse")
    data_detect = np.array(detrended_data)
    mask = np.ones(len(data_detect), dtype=bool)
    lacourse = detect_lacourse(
        data_detect,
        mask,
        sampling_rate=250,
    )

    print(f"Lacourse found {len(lacourse)} spindles")

    # Filter data online like on Portiloop
    print(f"Filtering Online with standardization")
    filtering_online = FilterPipeline(nb_channels=1, sampling_rate=250, filter_args=[True, True, True])
    filtered_online = []
    for i in tqdm(raw):
        filtered_online.append(filtering_online.filter(np.array([i])))
    filtered_online = np.array(filtered_online).flatten()

    return detrended_data, lacourse, filtered_online    


In [9]:
path = '/project/portinight-raw/PN_01_HJ/'
save_path = '/project/portinight-dataset/'
subject_id = 'PN_01_HJ'
age = 28
gender = 'M'

# Iterate through all the csvs in the folder
data = {}
for idx, file in enumerate(os.listdir(path)):
    if file.endswith(".csv"):
        print(f"Processing {file}")

        # Get the filename
        filename = file.split('.')[0] + f"_night{idx}"
        data[filename] = {}

        data[filename]['age'] = age
        data[filename]['gender'] = gender
        df = pd.read_csv(path + file, on_bad_lines='warn', encoding_errors='ignore')
        useful_data = df.iloc[:, 1].values
        filtered4lac, lacourse, filtered_online = raw2filtered(useful_data)

        spindle_info_mass = {}
        spindle_info_mass[filename] = {
            'onsets': [],
            'offsets': [],
            'labels_num': []
        }

        for spindle in lacourse:
            spindle_info_mass[filename]['onsets'].append(spindle[0])
            spindle_info_mass[filename]['offsets'].append(spindle[1])
            spindle_info_mass[filename]['labels_num'].append(1)

        data[filename]['signal_mass'] = filtered4lac
        data[filename]['signal_filt'] = filtered_online
        data[filename]['ss_label'] = np.ones(len(filtered4lac)) * 5
        data[filename]['spindle_mass_lacourse'] = spindle_info_mass

print(f"Saving f{subject_id}.npz")
np.savez_compressed(os.path.join(save_path, f"{subject_id}.npz"), data)
        

Processing PN_01_HJ_Night1_Stim.csv


Skipping line 1832260: expected 7 fields, saw 10

Skipping line 2995247: expected 7 fields, saw 9



Filtering Data Online


  0%|          | 0/3763765 [00:00<?, ?it/s]/tmp/ipykernel_540234/500185841.py:121: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  value[i] = x
100%|██████████| 3763765/3763765 [01:24<00:00, 44579.80it/s]


Detrending Data
Running Lacourse
Lacourse found 661 spindles
Filtering Online with standardization


...

100%|██████████| 5623560/5623560 [02:53<00:00, 32428.50it/s]


Processing PN_01_HJ_Night3_Stim.csv


Skipping line 820127: expected 7 fields, saw 11

Skipping line 2132013: expected 7 fields, saw 10



Filtering Data Online


  0%|          | 0/6373659 [00:00<?, ?it/s]/tmp/ipykernel_540234/500185841.py:121: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  value[i] = x
 12%|█▏        | 751073/6373659 [00:16<02:02, 45721.17it/s]